In [6]:
import json, datetime, math
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import scale
import operator
from sklearn.metrics import calinski_harabaz_score
from sklearn.cluster import KMeans
import wordcloud
import pickle
from sklearn.preprocessing import OneHotEncoder

In [44]:
tweet_train_labels = []
tweet_test_labels = []
with open("cluster_labels", "rb") as f:
    tweet_train_labels = pickle.load(f)
    tweet_test_labels = pickle.load(f)

In [45]:
tweet_train_labels = [[x] for x in tweet_train_labels]
tweet_test_labels = [[x] for x in tweet_test_labels]

In [42]:
tweet_train_labels = np.array(tweet_train_labels)
tweet_test_labels = np.array(tweet_test_labels)

In [53]:
ohe = OneHotEncoder()
ohe.fit(tweet_train_labels)
ohe_tweet_train_cluster_labels = ohe.transform(tweet_train_labels).toarray()
ohe_tweet_test_cluster_labels = ohe.transform(tweet_test_labels).toarray()

In [54]:
type(ohe_tweet_test_cluster_labels)

numpy.ndarray

In [86]:
import json, statistics, pprint, re
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from time import sleep
from datetime import datetime
import numpy as np
from pprint import pprint

with open("data/orig_en_train.json") as f:
    data = json.load(f)

split = len(data)//2
train = data[split:]
valid = data[:split]

X_train = []
y_train = []

account_likes = {}
tweets = {}
tweet_clients = defaultdict(int)
for d in train:
    tweet_clients[d['tweet_client_name']] += 1
    # tweet_text = d['tweet_text']
    # if tweet_text in tweets:
    #     tweets[tweet_text].append(int(d['like_count']))
    # else:
    #     tweets[tweet_text] = [int(d['like_count'])]
    user = d['userid']
    if user in account_likes:
        account_likes[user].append(int(d['like_count']))
    else:
        account_likes[user] = [int(d['like_count'])]
#     if int(d['like_count']) > 10:
#         pprint(d)
#         assert False


clients = {}
i = 0
for client, count in tweet_clients.items():
    clients[client] = i
    i += 1

for user, likes in account_likes.items():
    account_likes[user] = statistics.median(likes)

popular_hashtags = {'sports': 0, 'politics': 1, 'local': 2, 'world': 3, 'news': 4, 'SanDiego': 5, 'Syria': 6, \
                    'ISIS': 7, 'WakeUpAmerica': 8, 'Trump': 9, 'business': 10, 'Chicago': 11, 'health': 12, \
                    'love': 13, 'TopNews': 14, 'BLM': 15, 'PoliceBrutality': 16, 'Obama': 17, 'BlackTwitter': 18, \
                    'Trump2016': 19}

def process_tweet(tweet):
    if len(re.findall("(https://[^\s]+)", tweet)) > 0:
        tweet = re.sub("(https://[^\s]+)", "", tweet)
    if len(re.findall("(http://[^\s]+)", tweet)) > 0:
        tweet = re.sub("(http://[^\s]+)", "", tweet)
    if len(re.findall('@[^\s@]+', tweet)) > 0:
        tweet = re.sub('@[^\s@]+', "", tweet)
    if len(re.findall("(#[^#\s]+)", tweet)) > 0:
        tweet = re.sub("(#[^#\s]+)", "", tweet)
    return tweet

corpus = []
for d in train:
    corpus.append(process_tweet(d['tweet_text']))
    # corpus.append(d['tweet_text'])

bow = TfidfVectorizer(min_df=0.01, max_df=0.95)  #
bow.fit(corpus)
print(len(bow.vocabulary_))

def get_features(d, ind, data_type="train"):
    features = []

    # mean # of likes
    user =  d['userid']
    if user in account_likes:
        features.append(0)
        features.append(account_likes[user])
    else:
        features.append(1)
        features.append(0)

    # Twitter client
    client_features = [0 for i in range(len(clients))]
    client = d['tweet_client_name']
    if client in clients:
        features.append(0)
        client_index = clients[client]
        client_features[client_index] = 1
    else:
        features.append(1)
    features.extend(client_features)

    # popular hashtags
    curr_hashtags = d['hashtags'][1:-1].split(', ')
    hashtag_features = [0 for i in range(len(popular_hashtags))]
    for ht in curr_hashtags:
        if ht in popular_hashtags:
            ht_index = popular_hashtags[ht]
            hashtag_features[ht_index] = 1
    features.extend(hashtag_features)

    # url
    if 'https://t.co' in d['tweet_text'] or d['urls'] != '':
        features.append(1)
    else:
        features.append(0)

    # follower count
    features.append(int(d['follower_count']))

    # BOW
    features.extend(bow.transform([d['tweet_text']]).toarray()[0])

    # Time since epoch
    date = datetime.strptime(d['tweet_time'], '%Y-%m-%d %H:%M')
    features.append(date.timestamp())

    # Time of day
    minutes_in_day = 24 * 60
    minutes = date.hour * 60 + date.minute
    sin_time = np.sin(2 * np.pi * minutes / minutes_in_day)
    cos_time = np.cos(2 * np.pi * minutes / minutes_in_day)

    features.append(sin_time)
    features.append(cos_time)
    
    # cluster features
    if data_type == "train" or data_type == "valid":
        features.extend(ohe_tweet_train_cluster_labels[ind].tolist())
    else:
        features.extend(ohe_tweet_test_cluster_labels[ind].tolist())
    return features

104


In [87]:
X_train = []
y_train = []
for ind,d in enumerate(train):
    X_train.append(get_features(d, ind))
    if d['like_count'] == '0':
        y_train.append(0)
    else:
        y_train.append(1)

X_valid = []
y_valid = []
for ind, d in enumerate(valid):
    X_valid.append(get_features(d, split+ind))
    like_count = int(d['like_count'])
    if like_count == 0:
        y_valid.append(0)
    else:
        y_valid.append(1)

In [88]:
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)
X_valid = np.array(X_valid, dtype=np.float32)
y_valid = np.array(y_valid, dtype=np.float32)

In [89]:
X_train.shape

(25000, 187)

In [92]:
# best_score = 0
# best_lr = None
# possible_lrs = [0.1, 0.3, 0.5, 0.7, 1, 1.4, 1.6, 2.3, 4]
# for lr in possible_lrs:
ada = AdaBoostClassifier(learning_rate=1.6)
ada.fit(list(X_train), y_train)
print(ada.score(X_train, y_train))
valid_Score = ada.score(X_valid, y_valid)
    
#     if valid_Score > best_score:
#         best_score = valid_Score
#         best_lr = lr
print(valid_Score)
# print(best_lr)

0.85044
0.84604


In [ ]:
# follower count, url, client, cluster labels
# 0.83812
# 0.8356

# median likes, follower count, url, client, cluster labels
# 0.846
# 0.84128

# median likes, follower count, url, client, time since epoch, time of day, cluster labels
# 0.8496
# 0.84384

# median likes, top 10 hashtags, follower count, url, client, time since epoch, time of day, cluster labels
# 0.84972
# 0.84376

# BOW with min/max df, median likes, top 10 hashtags, follower count, url, client, 
# time since epoch, time of day, cluster labels
# 0.85044
# 0.84604